## 导入必要的包

In [1]:
from datasets import Dataset, load_dataset
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, BitsAndBytesConfig, GenerationConfig
import torch

In [2]:
print("torch version: ", torch.__version__)
print("transformers version: ", transformers.__version__)

torch version:  2.2.1+cu121
transformers version:  4.39.0


In [3]:
MAX_LENGTH = 2048    # 分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
data_path = "../data/emo_1234_xtuner.json"
pretrained_model_name_or_path = "../models/internlm2-chat-1_8b"
work_dir = "../work_dirs/internlm2_chat_1_8b_qlora_emo_e3_hf"

## 载入数据

In [4]:
ds = Dataset.from_json(data_path)
ds

Dataset({
    features: ['conversation'],
    num_rows: 16011
})

In [5]:
ds[0]

{'conversation': [{'input': '医生，我最近总是感到很焦虑，尤其是在学业上。我有个特别崇拜的同学，他好像在各方面都比我优秀，我总觉得自己怎么努力也追不上他，这让我压力特别大。',
   'output': '你好，首先感谢你对我敞开心扉。你的这种情绪其实很常见，这是由于过度比较和自我期待过高引发的焦虑情绪。我们要认识到每个人都有自己的发展节奏和优势，与他人比较并不是衡量自身价值的唯一标准。你可以试着列出自己在学习和其他方面的优点，同时理解并接纳自己的不足。我们可以一步步来，先从调整自我认知开始。',
   'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决。'},
  {'input': '是的，我知道应该理性看待，但就是忍不住会去比较。我甚至晚上会因为这个睡不着觉，总想着怎样才能像他那样出色。',
   'output': '了解你的情况后，我建议你在睡前尝试进行放松训练，比如深呼吸、冥想等方法，有助于改善睡眠质量。另外，设定实际可达的目标也是很重要的，避免给自己设定过于严苛的标准。你是否可以具体描述一下你在学业上的困扰，我们一起看看如何制定一个适合你的个人提升计划？',
   'system': None},
  {'input': '我在数学方面总是比他落后很多，我尝试了很多方法提高，但还是觉得力不从心。',
   'output': '对于数学学习困难的问题，我们可以探讨更具针对性的学习策略，如分解复杂问题为小块、寻找优质的学习资源或参加学习小组等。同时，也要建立对失败的正确认识，失败是成功的垫脚石，每一次错误都是我们进步的机会。接下来，我会推荐一些有效的学习技巧，并鼓励你定期与我分享进展，以便我们根据实际情况进行调整。\n医生：另外，请记住，寻求帮助也是一种能力，如果你感到持续的压力和焦虑影响了生活和学习，不妨考虑找学校的心理辅导老师或者进一步接受专业心理咨询。心理健康与学业成就同样重要，让我们一起关注并呵护它。',
   'system': None}]}

## 处理数据集

In [6]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path, use_fast=False, trust_remote_code=True)
tokenizer

InternLM2Tokenizer(name_or_path='../models/internlm2-chat-1_8b', vocab_size=92544, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|action_start|>', '<|action_end|>', '<|interpreter|>', '<|plugin|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	92538: AddedToken("<|plugin|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	92539: AddedToken("<|interpreter|>", rstrip=False, lstrip=False, single_word=False, norm

In [7]:
tokenizer.all_special_ids

[1, 2, 0, 92543, 92542, 92541, 92540, 92539, 92538]

In [8]:
tokenizer.all_special_tokens

['<s>',
 '</s>',
 '<unk>',
 '<|im_start|>',
 '<|im_end|>',
 '<|action_start|>',
 '<|action_end|>',
 '<|interpreter|>',
 '<|plugin|>']

In [9]:
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id

(1, 2, 2)

In [10]:
tokenizer.encode("<s>[UNUSED_TOKEN_146][UNUSED_TOKEN_145]", add_special_tokens=False)

[1, 92543, 92542]

In [11]:
tokenizer.decode([1, 92543, 92542], skip_special_tokens=False)

' <s><|im_start|><|im_end|>'

In [12]:
tokenizer.encode("<s><|im_start|><|im_end|>", add_special_tokens=False)

[1, 92543, 92542]

In [13]:
# https://github.com/InternLM/xtuner/blob/main/xtuner/utils/templates.py#L24
internlm2_chat = dict(
    SYSTEM = '<|im_start|>system\n{system}<|im_end|>\n',
    INSTRUCTION = ('<|im_start|>user\n{input}<|im_end|>\n'
                   '<|im_start|>assistant\n'),
    SUFFIX = '<|im_end|>',
    SUFFIX_AS_EOS = True,
    SEP = '\n',
    STOP_WORDS = ['<|im_end|>'])

In [14]:
# https://huggingface.co/internlm/internlm2-chat-1_8b/blob/main/modeling_internlm2.py#L1136-L1146
def build_inputs(
    tokenizer,
    query: str,
    history: list[tuple[str, str]] | None = None,
    meta_instruction = ""
) -> tuple[str, list]:
    history = [] if history is None else list(history)
    if tokenizer.add_bos_token:
        prompt = ""
    else:
        prompt = tokenizer.bos_token
    if meta_instruction:
        prompt += f"""<|im_start|>system\n{meta_instruction}<|im_end|>\n"""
    for record in history:
        prompt += f"""<|im_start|>user\n{record[0]}<|im_end|>\n<|im_start|>assistant\n{record[1]}<|im_end|>\n"""
    prompt += f"""<|im_start|>user\n{query}<|im_end|>\n<|im_start|>assistant\n"""
    return prompt, tokenizer([prompt], return_tensors="pt")

In [15]:
build_inputs('你好')

'<s><|im_start|>system\n我是系统<|im_end|>\n<|im_start|>user\n你好<|im_end|>\n<|im_start|>assistant\n'

In [16]:
def process_func(example):
    # print(example)
    # {
    #    'conversation': [
    #         {
    #             'input': '医生，我最近总是感到很焦虑，尤其是在学业上。我有个特别崇拜的同学，他好像在各方面都比我优秀，我总觉得自己怎么努力也追不上他，这让我压力特别大。\n\n',
    #             'output': '你好，首先感谢你对我敞开心扉。你的这种情绪其实很常见，这是由于过度比较和自我期待过高引发的焦虑情绪。我们要认识到每个人都有自己的发展节奏和优势，与他人比较并不是衡量自身价值的唯一标准。你可以试着列出自己在学习和其他方面的优点，同时理解并接纳自己的不足。我们可以一步步来，先从调整自我认知开始。\n\n',
    #             'system': '现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决。'
    #         },
    #         {
    #             'input': '是的，我知道应该理性看待，但就是忍不住会去比较。我甚至晚上会因为这个睡不着觉，总想着怎样才能像他那样出色。\n\n',
    #             'output': '了解你的情况后，我建议你在睡前尝试进行放松训练，比如深呼吸、冥想等方法，有助于改善睡眠质量。另外，设定实际可达的目标也是很重要的，避免给自己设定过于严苛的标准。你是否可以具体描述一下你在学业上的困扰，我们一起看看如何制定一个适合你的个人提升计划？\n\n',
    #             'system': None
    #         }
    #     ]
    # }

    # 开始的 bos token
    input_ids = [tokenizer.bos_token_id]
    attention_mask = [1]
    labels = [-100]

    # 多轮对话
    for i, conversation in enumerate(example['conversation']):
        # 第一轮添加system指令
        if i == 0:
            text = f"<|im_start|>system\n{conversation['system']}<|im_end|>\n<|im_start|>user\n{conversation['input']}<|im_end|>\n<|im_start|>assistant\n"
        else:
            text = f"<|im_start|>user\n{conversation['input']}<|im_end|>\n<|im_start|>assistant\n"
        instruction = tokenizer(text, add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
        response = tokenizer(f"{conversation['output']}<|im_end|>\n", add_special_tokens=False)
        input_ids += instruction["input_ids"] + response["input_ids"]
        attention_mask += instruction["attention_mask"] + response["attention_mask"]
        labels += [-100] * len(instruction["input_ids"]) + response["input_ids"]

    # 结束的 eos token
    input_ids += [tokenizer.eos_token_id]
    attention_mask += [1]                   # 因为eos token咱们也是要关注的所以 补充为1
    labels += [tokenizer.eos_token_id]

    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [28]:
# remove_columns: map 后会移除这一列
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 16011
})

In [29]:
ds.column_names

['conversation']

In [30]:
print(tokenized_id[0].keys())
print(tokenized_id[0]['input_ids'])
print(tokenized_id[0]['attention_mask'])
print(tokenized_id[0]['labels'])

dict_keys(['input_ids', 'attention_mask', 'labels'])
[1, 92543, 9081, 364, 68293, 60403, 68625, 68831, 69212, 60353, 60363, 71525, 68831, 68287, 60353, 78463, 60379, 70159, 68580, 72863, 68530, 60355, 92542, 364, 92543, 1008, 364, 69156, 60353, 60363, 68848, 68965, 69836, 60427, 71833, 60353, 78931, 77649, 60370, 60355, 60363, 69054, 68425, 78126, 72246, 60353, 60404, 69123, 60361, 77801, 60406, 72256, 68877, 60353, 60363, 60649, 70868, 68288, 68626, 60395, 61286, 69842, 60404, 60353, 60376, 68678, 68912, 68425, 60368, 60355, 92542, 364, 92543, 525, 11353, 364, 77230, 60353, 68400, 69894, 60403, 69880, 63612, 69166, 64886, 60355, 68364, 68381, 69209, 68377, 60427, 69305, 60353, 68472, 68560, 71631, 68324, 60381, 69396, 70271, 76436, 90530, 71833, 69209, 60355, 69897, 80966, 78747, 68304, 68357, 71228, 60381, 69203, 60353, 60510, 70535, 68324, 69461, 77546, 69238, 76906, 69362, 68559, 60355, 69686, 75630, 76048, 73799, 68352, 71842, 69834, 71280, 60353, 68405, 68865, 60573, 82567, 68304

In [31]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))

 <s><|im_start|> system
现在你是一个心理专家，我有一些心理问题，请你用专业的知识帮我解决。<|im_end|> 
<|im_start|> user
医生，我最近总是感到很焦虑，尤其是在学业上。我有个特别崇拜的同学，他好像在各方面都比我优秀，我总觉得自己怎么努力也追不上他，这让我压力特别大。<|im_end|> 
<|im_start|> assistant
你好，首先感谢你对我敞开心扉。你的这种情绪其实很常见，这是由于过度比较和自我期待过高引发的焦虑情绪。我们要认识到每个人都有自己的发展节奏和优势，与他人比较并不是衡量自身价值的唯一标准。你可以试着列出自己在学习和其他方面的优点，同时理解并接纳自己的不足。我们可以一步步来，先从调整自我认知开始。<|im_end|> 
<|im_start|> user
是的，我知道应该理性看待，但就是忍不住会去比较。我甚至晚上会因为这个睡不着觉，总想着怎样才能像他那样出色。<|im_end|> 
<|im_start|> assistant
了解你的情况后，我建议你在睡前尝试进行放松训练，比如深呼吸、冥想等方法，有助于改善睡眠质量。另外，设定实际可达的目标也是很重要的，避免给自己设定过于严苛的标准。你是否可以具体描述一下你在学业上的困扰，我们一起看看如何制定一个适合你的个人提升计划？<|im_end|> 
<|im_start|> user
我在数学方面总是比他落后很多，我尝试了很多方法提高，但还是觉得力不从心。<|im_end|> 
<|im_start|> assistant
对于数学学习困难的问题，我们可以探讨更具针对性的学习策略，如分解复杂问题为小块、寻找优质的学习资源或参加学习小组等。同时，也要建立对失败的正确认识，失败是成功的垫脚石，每一次错误都是我们进步的机会。接下来，我会推荐一些有效的学习技巧，并鼓励你定期与我分享进展，以便我们根据实际情况进行调整。
医生：另外，请记住，寻求帮助也是一种能力，如果你感到持续的压力和焦虑影响了生活和学习，不妨考虑找学校的心理辅导老师或者进一步接受专业心理咨询。心理健康与学业成就同样重要，让我们一起关注并呵护它。<|im_end|> 
</s>


In [32]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

' 你好，首先感谢你对我敞开心扉。你的这种情绪其实很常见，这是由于过度比较和自我期待过高引发的焦虑情绪。我们要认识到每个人都有自己的发展节奏和优势，与他人比较并不是衡量自身价值的唯一标准。你可以试着列出自己在学习和其他方面的优点，同时理解并接纳自己的不足。我们可以一步步来，先从调整自我认知开始。<|im_end|> \n了解你的情况后，我建议你在睡前尝试进行放松训练，比如深呼吸、冥想等方法，有助于改善睡眠质量。另外，设定实际可达的目标也是很重要的，避免给自己设定过于严苛的标准。你是否可以具体描述一下你在学业上的困扰，我们一起看看如何制定一个适合你的个人提升计划？<|im_end|> \n对于数学学习困难的问题，我们可以探讨更具针对性的学习策略，如分解复杂问题为小块、寻找优质的学习资源或参加学习小组等。同时，也要建立对失败的正确认识，失败是成功的垫脚石，每一次错误都是我们进步的机会。接下来，我会推荐一些有效的学习技巧，并鼓励你定期与我分享进展，以便我们根据实际情况进行调整。\n医生：另外，请记住，寻求帮助也是一种能力，如果你感到持续的压力和焦虑影响了生活和学习，不妨考虑找学校的心理辅导老师或者进一步接受专业心理咨询。心理健康与学业成就同样重要，让我们一起关注并呵护它。<|im_end|> \n</s>'

## 创建模型

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # 是否在4位精度下加载模型。如果设置为True，则在4位精度下加载模型。
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float16,   # 4位精度计算的数据类型。这里设置为torch.float16，表示使用半精度浮点数。
    bnb_4bit_quant_type='nf4',              # 4位精度量化的类型。这里设置为"nf4"，表示使用nf4量化类型。 nf4: 4bit-NormalFloat
    bnb_4bit_use_double_quant=True,         # 是否使用双精度量化。如果设置为True，则使用双精度量化。
)
quantization_config

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map='auto',
    low_cpu_mem_usage=True,             # 是否使用低CPU内存，使用 device_map 参数必须为 True
    quantization_config=quantization_config,
)
model.enable_input_require_grads()      # 开启梯度检查点时，要执行该方法
model

In [ ]:
print(f"model.device: {model.device}, model.dtype: {model.dtype}")

## Lora

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training, load_peft_weights

In [ ]:
# https://huggingface.co/docs/peft/developer_guides/quantization
model = prepare_model_for_kbit_training(model)
model

In [ ]:
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,   # 训练模式
    r=64,                   # Lora 秩
    target_modules=['wqkv', 'wo', 'w1', 'w2', 'w3'],
    lora_alpha=16,          # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,       # Dropout 比例
    bias='none'
)
config

In [ ]:
model = get_peft_model(model, config)
config

In [ ]:
model.print_trainable_parameters()

## 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir=work_dir,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    logging_steps=10,
    save_strategy="epoch",  # epoch or steps
    save_steps=1,           # 每个epoch保存一次模型
    save_total_limit=3,
    save_on_each_node=True,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    bf16 = False,   # 指定训练时的类型
    fp16 = True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
trainer.train()